# Spectral-based GNN     

## 基础概念

<img style="float: center;" src="resource_pic/GNN/GNN_1_11.PNG" width=600 height=600>

- **拓扑图卷积的方法:**

1. 把node的feature看作signal   


2. 对signal和filter都做傅里叶变换    


3. 卷积——即相乘


4. 傅里叶逆变换     

- **合成与分析——傅里叶变换**             

信号是一个$N$维空间里的一个向量，这个向量由$N$个$basis——\hat v_k$线性加权组成:

假设这些$basis$是**标准正交的**(orthonormal)的$basis$，要计算线性组合的权重，只需要将最终的**signal向量**和**基向量**做内积    


$$\vec{A} = \sum ^N_{k=1}a_k\hat v_k\ ——(合成)$$

$$
a_j = \vec{A}\cdot \hat v_j\ ——(分析)\\
\hat v_i \cdot\hat v_j = \sigma _{i,j}
$$

- **信号在时域与频域中的表示**       

一个信号可以用不同的方式来合成，如下图，在时域当中$\delta (t-\tau)$是一组$basis$              

傅里叶变换中(频域当中)，$e^{j\omega t}$是一组基,$X(j\omega)$是对应的系数

<img style="float: center;" src="resource_pic/GNN/GNN_1_12.PNG" width=600 height=600>    

当我们需要知道$X(j\omega)$是多少时，用如下公式，其中左式是一整个**分析的过程**               

<img style="float: center;" src="resource_pic/GNN/GNN_1_13.PNG" width=600 height=600>




## GCN原理      

对于图$G=(V,E),N = |V|$



1.  $A\in R^{N\times N}$:**邻接矩阵**(adjancency matrix/weight matrix)

    $A_{ij}=0,if\ e_{i,j}\notin E,else\ A_{ij} = w(i,j)$

    **仅考虑无向图**——$A$是一个对称矩阵
    
    
2. $D\in R^{N\times N}$:**度矩阵**(degree matrix),对角线上元素依次为各个顶点的度

    $$
    D_{ij}=\begin{cases}
    0,if \ i\neq j\\
    d(i),if \ i= j
    \end{cases}
    $$
    
    
3. $f:V\rightarrow R^{N\times N}$:一个映射，**代表graph的vertices上的信号**,$f(i)$是$v_i$上的信号
 
 <img style="float: center;" src="resource_pic/GNN/GNN_1_14.PNG" width=600 height=600>
 
 
### 拉普拉斯矩阵

对于图$G=(V,E)$其Laplacian 矩阵的定义为 $L=D−A$，其中$L$是$Laplacian$ 矩阵                  

拉普拉斯矩阵是**对称的半正定矩阵**

常用的拉普拉斯矩阵有三种：

- Combinatorial Laplacian：$L=D−A$
- Symmetric normalized Laplacian：$L^{sys}=D^{−1/2}LD^{−1/2}$
- Random walk normalized Laplacian：$L^{rw}=D^{−1}L$


### 为什么GCN要用拉普拉斯矩阵？

1. 拉普拉斯矩阵是**对称矩阵**，可以进行**特征分解（谱分解）**，和GCN的spectral domain对应


2. 拉普拉斯矩阵只在中心顶点和一阶相连的顶点上（1-hop neighbor）有非0元素，其余之处均为0


3. 通过拉普拉斯算子与拉普拉斯矩阵进类比           

### 拉普拉斯矩阵的谱分解（特征分解）

矩阵的**谱分解，特征分解，对角化**是同一个概念。 可以特征分解的充要条件：**为n阶方阵存在n个线性无关的特征向量**。

拉普拉斯矩阵是**半正定对称矩阵**（半正定矩阵本身就是对称矩阵）有如下三个性质：                   

    1. 对称矩阵一定n个线性无关的特征向量

    2. 半正定矩阵的特征值一定非负

    3. 对阵矩阵的特征向量相互正交，即所有特征向量构成的矩阵为正交矩阵。                                 
    
    
**拉普拉斯矩阵谱分解的结果——**      

$$
L = U\Lambda U^{-1}
=U\Lambda U^T
$$

其中，$U=(\vec u_1,\vec u_2,\cdots,\vec u_n)$，是列向量为单位特征向量的矩阵，由于$U$是正交矩阵，即$UU^T=E$。

$$
\Lambda = \left[
\begin{matrix}
\lambda_1 & 0 & \cdots & 0\\
0 & \lambda_2 & \cdots & 0\\
\vdots & \vdots & \ddots & \vdots\\
0 & 0 & \cdots & \lambda_n
\end{matrix}
\right]
$$
$\Lambda$是$n$个特征值构成的对角阵,且每个**特征值都大于零**。

**其中$\lambda_l$叫做频率(frequency),$u_l$是$\lambda_l$对应的$basis$**



<img style="float: center;" src="resource_pic/GNN/GNN_1_15.PNG" width=600 height=600>

## A case

### 不同频域下的信号能量差异     

-  以一个$vertex$上的特征为纯量(非向量)的图——则其拉普拉斯矩阵及其谱分解如下图所示:

其中$\lambda_i$代表的是不同$basis——u_i$下的频率，其中**频率越大，两点之间的差距也就越大**——可以通过离散傅里叶变换理解

<img style="float: center;" src="resource_pic/GNN/GNN_1_16.PNG" width=500 height=500>

如下图所示，每张子图分别代表一个频率域内的基函数，频率越大的基函数相邻两个采样点之间的信号差异越大

<img style="float: center;" src="resource_pic/GNN/GNN_1_20.PNG" width=500 height=500>


在特征值(频率)从低到高所对应的特征向量(基向量)中，$\lambda_0 \rightarrow \lambda_3$每个$u_i$点与点之间的**能量差异越大**
<img style="float: center;" src="resource_pic/GNN/GNN_1_17.PNG" width=500 height=500>


### 图的合成与分析          

定义拉普拉斯矩阵$L$为一个$G$上的拉普拉斯算子——a operator

对图$G$做如下运算:    

$$Lf = (D-A)f = Df-Af$$

如图所示，假设$Lf = [a,b,c,d]^T$，可以看到实际上$a = \sum_j (D_{1j}f - A_{1j}f)$

在度矩阵$D$中，只有$v_i$本身对应的位置的weight不为0——$d(i)$

在邻接矩阵$A$中，只有$v_i$邻接节点位置的weight不为0——$1$




<img style="float: center;" src="resource_pic/GNN/GNN_1_18.PNG" width=600 height=600>

那么$a = \sum_j (D_{1j}f - A_{1j}f)$代表**该节点的能量与邻接节点能量的差异**          

为了衡量差异的大小，需要对该差异平方，即$f^TLf$

<img style="float: center;" src="resource_pic/GNN/GNN_1_19.PNG" width=500 height=500>


## Graph的傅里叶变换与逆变换        


- 傅里叶变换    

一组拉普拉斯矩阵的$basis$是$U^T$,那么将信号$x$转入频域中——$\hat x = U^Tx$

其中$u_i\dot x$代表的是信号$x$在不同频域当中的强度
<img style="float: center;" src="resource_pic/GNN/GNN_1_21.PNG" width=600 height=600>


- 傅里叶逆变换   

一组频域内的信号转化为时域内信号——$x = U \hat x$

$U$为正交矩阵，因此$U^T = U^{-1}$
<img style="float: center;" src="resource_pic/GNN/GNN_1_22.PNG" width=600 height=600>

## Graph的卷积     

在时域上的卷积即在频域上的乘积    

定义一个映射$g_\theta:\Lambda \rightarrow \Theta$,即$\Theta = g(\Lambda),\theta_i = g_\theta(\lambda_i)$            

该映射函数$g_\theta$是需要学习的函数——Filter。   

### 频域上的卷积操作

$$\hat y = g_\theta(\Lambda)\hat x$$

<img style="float: center;" src="resource_pic/GNN/GNN_1_23.PNG" width=600 height=600>


### 卷积后还原到时域

$$y =U g_\theta(\Lambda)\hat x = U g_\theta(\Lambda)U^T x$$

<img style="float: center;" src="resource_pic/GNN/GNN_1_24.PNG" width=600 height=600>

## Spectral-based GNN的问题     

1. **计算复杂度及可拓展性的问题**      

Filter的大小和输入的图$G$大小有关——要学习$N$个$\theta$值，与CNN不同的是，CNN无论输入的图多大，卷积核学习的都只是固定数量的参数。     

Spectral-based GNN的计算复杂度为$o(N)$

2. **可能会在学习过程中忽略局部特征**            

$$y =U g_\theta(\Lambda)\hat x = U g_\theta(\Lambda)U^T x = g_\theta(L)x$$

那么，假设$g_\theta(L) = L^n,n\geq 2$时，会导致$G$中不相连的元素在矩阵中对应元素**不为0**，即认为两者相互可到达，于是可能会丧失局部化的能力。